In [ ]:
!pip install requests # for retrieiving data

In [ ]:
from tandv.track.common import read_pickle, TrainingStats
from tandv.viz import exp_hist,scalar_global_heatmap,scalar_line,interactive
import pickle,requests

## Loading Example Data

In [ ]:
# Download sample data from s3
lf_url = 'https://graphcore-research-public.s3.eu-west-1.amazonaws.com/tandv/sampleLogFrame.pkl'
ts_url = 'https://graphcore-research-public.s3.eu-west-1.amazonaws.com/tandv/sampleTrainingStats.pkl'
lf = read_pickle(lf_url)
res = requests.get(ts_url)
res.raise_for_status()
tstats = pickle.loads(res.content)


## Interactive Exponent Histogram
Pass `exp_hist`function into `interactive` along with valid arguements to generate an initial plot.

You can then use the various widgets to query along different dimensions (step, layer, tensor type, etc..) and subsequently generate visualizations.

In [ ]:
interactive(exp_hist,
    df=lf,
    layer='output',
    tt = 'Activation',
    step = 0,
    dtype_annotation = 'float8_e4m3fn'
)

## Interactive Scalar Line 
Pass `scalar_line`function into `interactive` along with valid arguements to generate an initial plot.

You can then use the various widgets to query along different dimensions (scalar_metric, layer, tensor type, etc..) and subsequently generate visualizations.

In [ ]:
interactive(
    scalar_line,
    df=lf,
    layer='output',
    tt='Gradient',
    scalar_metric='std',
)

## Interactive Global Heatmap 
Pass `scalar_global_heatmap`function into `interactive` along with valid arguements to generate an initial plot.

You can then use the various widgets to query along different dimensions (scalar_metric, layer, tensor type, etc..) and subsequently generate visualizations.

You can also click on the heatmap patches and it will generate an `exp_hist` vizualization along those query dimensions (step, tensor_type and layer)

In [ ]:
interactive(
    scalar_global_heatmap,
    df=lf,
    tt='Optimiser_State.exp_avg',
    scalar_metric='rm2',
    inc=50,
)

# Cross Referencing with Training Stats

Query exponent histograms from the loss curve

In [ ]:
interactive(exp_hist,
    train_stats=tstats,
    df=lf,
    layer=[n for n in lf.metadata.name.unique().tolist() if 'layers.5.feed_forward.w' in n],
    tt = 'Activation',
    step = 0,
    dtype_annotation = 'float8_e4m3fn',
    col_wrap = 3
)

Cross referencing loss curve(s) with scalar statistics of various tensors in the network

In [ ]:
interactive(
    scalar_line,
    train_stats=tstats,
    mouse_sensitivity=20000,
    df=lf,
    layer=[n for n in lf.metadata.name.unique().tolist() if 'layers.5.feed_forward.w' in n],
    tt='Gradient',
    scalar_metric=['std','rm2'],
    col_wrap = 3
)